README

This code is for model comparison using the 12320x176 matrices inferred by each model applied to the test dataset (n=176).

1. Input 12320x176 matrix should be .pkl file containing numpy matrix of shape (12320, 176).
It assumes that all data are in same scale, from 0 to 1. As of now (9th Dec 2024), the input .pkl files are stored in: Harim/Analysis/Test_dataset_inferred

2. model_summary and lmid_sumamry files. These files are used to create dictionaries, which help us to match the model and its information (such as number of landmarks used, landmark ids, etc).

In [3]:
# import required modules

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import numpy as np
import pandas as pd
import pickle
from cmapPy.pandasGEXpress.parse import parse
import scipy.stats as stats
import torch
import torch.nn as nn

def load_gene_ids(lmid_txt):
  with open(lmid_txt, 'r') as f: lmid = [l[:-1] for l in f.readlines()]
  return lmid

In [ ]:
# Feature selected landmark ids (with LASSO)

lmid_alpha_005 = ['3800'] # for LASSO 1
lmid_alpha_003 = ["6616", "10398", "6275", "1759", "1277", "3800", "5909", "230", "2810", "4638"] # for LASSO 10
lmid_alpha_001 = ["5997", "427", "5054", "25987", "23670", "7466", "11182", "3385", "56654", "5696",
                   "5588", "2185", "3315", "8553", "6616", "7849", "7494", "991", "6347", "6813",
                   "11065", "3337", "3157", "25932", "10398", "4860", "1050", "10776", "949", "7168",
                   "79170", "10610", "9455", "5480", "8349", "6275", "6915", "9170", "7867", "25825",
                   "1759", "54541", "30001", "348", "29763", "85236", "3725", "55620", "1831", "5627",
                   "3202", "56924", "2690", "3566", "3815", "5788", "27346", "5357", "6195", "79921",
                   "23321", "11261", "23530", "1277", "1958", "4582", "1052", "7153", "4616", "11041",
                   "8851", "6696", "2353", "4609", "3775", "3206", "2064", "8324", "6919", "2274",
                   "3800", "3162", "1282", "2037", "10810", "1026", "11151", "10797", "976", "3383",
                   "2625", "23659", "10857", "9289", "1514", "5154", "4783", "727", "9805", "2920",
                   "11098", "56997", "823", "6812", "3693", "3122", "5909", "1123", "3486", "5603",
                   "10221", "9133", "79961", "5836", "960", "2065", "7074", "10099", "5468", "664",
                   "9531", "230", "7106", "1978", "7852", "5777", "84617", "11031", "2745", "3280",
                   "5236", "4016", "4651", "26227", "8870", "2810", "10276", "4638", "813", "481",
                   "291", "5831"] # for LASSO 142
lmid_lasso_108 = ['5997', '5054', '23670', '7466', '3385', '56654', '5696', '2185',
       '3315', '8553', '6616', '7849', '7494', '991', '6347', '6813',
       '11065', '3337', '3157', '25932', '10398', '4860', '1050', '10776',
       '949', '7168', '79170', '10610', '9455', '5480', '6275', '6915',
       '7867', '1759', '54541', '348', '29763', '1831', '3202', '56924',
       '2690', '3566', '3815', '5788', '27346', '5357', '6195', '11261',
       '23530', '1277', '1958', '4582', '1052', '4616', '8851', '6696',
       '2353', '4609', '3775', '3206', '2064', '2274', '3800', '3162',
       '1282', '1026', '11151', '10797', '976', '2625', '10857', '9289',
       '4783', '9805', '2920', '11098', '56997', '823', '6812', '3693',
       '3122', '5909', '1123', '3486', '5603', '9133', '5836', '960',
       '2065', '7074', '664', '9531', '230', '7106', '1978', '7852',
       '5777', '84617', '2745', '3280', '5236', '26227', '8870', '2810',
       '10276', '4638', '481', '291'] # for LASSO 108

# Random subsampling (slicing by index)
lmid_970 = load_gene_ids("lmid_970.txt") # check your "lmid_970.txt" path
lmid_486 = lmid_970[:486]
lmid_324 = lmid_970[:324]
lmid_108 = lmid_970[:108]
lmid_10 = lmid_970[:10]
lmid_1 = lmid_970[:1]

# Greedy forward subsampling (slicing by index)
gf_108 = load_gene_ids("greedy_forward_id.txt")   # check your "greedy_forward_id.txt" path
gf_10 = gf_108[:10]
gf_1 = gf_108[:1]


In [ ]:
# dictionary: key, (filename, lmid)
models = pd.read_table("/your/model/summary/path", dtype="S")  # check your model summary file path
lmids = pd.read_table("/your/landmark/id/list/path", dtype="S") # check your lmid_summary file path
lmids['lmid_list'] = pd.Series([lmid_lasso_108, lmid_alpha_001, lmid_alpha_003, lmid_alpha_005,
                                lmid_486, lmid_324, lmid_108, lmid_10, lmid_1, gf_108, gf_10, gf_1, lmid_970, lmid_970]) # if the lmid_summary file is modified, change this line accordingly
lmids = lmids.set_index(['feature_selection_method','num_landmarks'])
display(lmids)

lmid_list
feature_selection_method num_landmarks                                                   
LASSO                    108            [5997, 5054, 23670, 7466, 3385, 56654, 5696, 2...
                         142            [5997, 427, 5054, 25987, 23670, 7466, 11182, 3...
                         10             [6616, 10398, 6275, 1759, 1277, 3800, 5909, 23...
                         1                                                         [3800]
random                   486            [5720, 7416, 55847, 10174, 25803, 466, 6676, 1...
                         324            [5720, 7416, 55847, 10174, 25803, 466, 6676, 1...
                         108            [5720, 7416, 55847, 10174, 25803, 466, 6676, 1...
                         10             [5720, 7416, 55847, 10174, 25803, 466, 6676, 1...
                         1                                                         [5720]
greedy                   108            [3800, 4303, 7168, 2810, 5641, 813, 291, 3930,...
                         10             [3800, 4303, 7168, 2810, 5641, 813, 291, 3930,...
                         1                                                         [3800]
all                      970            [5720, 7416, 55847, 10174, 25803, 466, 6676, 1...
                         978            [5720, 7416, 55847, 10174, 25803, 466, 6676, 1...

In [8]:
models

,filename,model,feature_selection_method,num_landmarks
0,GNN_all_970.pkl,GNN,all,970
1,GNN_greedy_1.pkl,GNN,greedy,1
2,GNN_greedy_10.pkl,GNN,greedy,10
3,GNN_greedy_108.pkl,GNN,greedy,108
4,GNN_LASSO_1.pkl,GNN,LASSO,1
5,GNN_LASSO_10.pkl,GNN,LASSO,10
6,GNN_LASSO_108.pkl,GNN,LASSO,108
7,GNN_random_1.pkl,GNN,random,1
8,GNN_random_10.pkl,GNN,random,10
9,GNN_random_108.pkl,GNN,random,108


In [ ]:
import numpy as np
import pandas as pd
import os
import scipy.stats as stats
import torch
import pickle
from scipy.stats import norm
from sklearn.metrics import roc_auc_score

def calculate_Error(GT, INF):
  # GT / INF : (12320, 176) numpy ndarray (row, column orders should match)
  # output: list of Error values (length: 176)
  err = []
  for i in range(INF.shape[1]):
    err.append(np.sqrt(np.sum((INF[:,i]-GT[:,i])**2)))
  return err

def calculate_SCC(GT, INF):
  # GT / INF : (12320, 176) numpy ndarray (row, column orders should match)
  # output: list of SCC values (length: 176)
  from scipy.stats import spearmanr
  scc = []
  for i in range(INF.shape[1]):
    scc.append(spearmanr(GT[:,i].flatten(), INF[:,i].flatten())[0])
  return scc

def calculate_PCC(GT, INF):
  # GT / INF : (12320, 176) numpy ndarray (row, column orders should match)
  # output: list of PCC values (length: 176)
  from scipy.stats import pearsonr
  pcc = []
  for i in range(INF.shape[1]):
    pcc.append(pearsonr(GT[:,i].flatten(), INF[:,i].flatten())[0])
  return pcc

def calculate_SMAPE(GT, INF):
  smape = []
  for i in range(INF.shape[1]):
    numerator = np.abs(GT[:, i] - INF[:, i])
    denominator = (np.abs(GT[:, i]) + np.abs(INF[:, i])) / 2
    denominator = np.where(denominator == 0, 1e-15, denominator)

    smape_value = np.mean((numerator / denominator) * 100)
    smape.append(smape_value)

  return smape

def recall_test(GT, INF, lmid, nonlmid, num_lm=970, outfn=None):
    # Load gene information for annotation
    gene_info = pd.read_csv("/content/geneinfo_beta.txt", sep="\t", dtype={"gene_id": str}) #check your "geneinfo_beta.txt" path
    gene_info_dict = gene_info.set_index("gene_id")["gene_symbol"].to_dict()

    GT_rank = stats.rankdata(GT, axis=1)
    INF_rank = stats.rankdata(INF, axis=1)

    recallcorr = np.corrcoef(GT_rank, INF_rank)[:12320, 12320:]
    self_corr = np.diagonal(recallcorr).tolist()

    np.fill_diagonal(recallcorr, np.nan)
    recallcorr = recallcorr[~np.isnan(recallcorr)].tolist()
    null_dist = recallcorr

    def calc_recall(null_dist, self_corr, qval):  
        # To reproduce recall = 99.9 self-correlation, modify this definition
        r_above_threshold = []
        threshold = np.percentile(null_dist, q=qval)
        for i in range(len(self_corr)):
            if self_corr[i] >= threshold:
                r_above_threshold.append(i)
        return r_above_threshold, len(r_above_threshold), threshold

    r95_list, r95, r95_threshold = calc_recall(null_dist, self_corr, 95)
    r99_list, r99, r99_threshold = calc_recall(null_dist, self_corr, 99)

    print(f"Among {len(self_corr)} inferred genes:")
    print(f"Rgene>0.95: {r95} ({r95 / len(self_corr) * 100:.2f}%)")
    print(f"Rgene>0.99: {r99} ({r99 / len(self_corr) * 100:.2f}%)")    

    # Create a null distribution and determine the percentile ranks of the self-correlation values.
    null_dist_np = np.sort(null_dist)
    positions = np.searchsorted(null_dist_np, self_corr, side='right')
    percentiles = (positions / len(null_dist_np)) * 100

    all_gene_symbols = [gene_info_dict.get(str(gene), str(gene)) for gene in lmid + nonlmid]
    all_self_corr = self_corr
    all_r95_flags = [1 if i in r95_list else 0 for i in range(len(lmid + nonlmid))]
    all_r99_flags = [1 if i in r99_list else 0 for i in range(len(lmid + nonlmid))]
    is_lmid = [1 if i in lmid else 0 for i in (lmid + nonlmid)]


    recall_df_all = pd.DataFrame({
        "Gene_Symbol": all_gene_symbols,
        "Gene_ID": lmid + nonlmid,
        "Self_Correlation": all_self_corr,
        "Recall": percentiles,
        "Recall_95": all_r95_flags,
        "Recall_99": all_r99_flags,
        "is_lmid": is_lmid
    })
    if not os.path.exists("BING"): os.makedirs("BING")
    recall_df_all.to_csv("BING/" + f"{outfn}_recall_result.csv", index=False)
    return r95, r99


def calculate_AUROC(GT, INF):
    # GT / INF : (12320, 176) numpy ndarray to tensor
    # output: mean and std
    GT_tensor= torch.tensor(GT)
    INF_tensor= torch.tensor(INF)
    median_values = torch.median(GT_tensor, dim=0).values
    binary_labels = (GT_tensor > median_values).int()

    auroc_scores = []
    for i in range(176):
        y_true = binary_labels[:, i].numpy().ravel()
        y_score = INF_tensor[:, i].numpy().ravel()
        y_score = (y_score - y_score.min()) / (y_score.max() - y_score.min())
        auroc = roc_auc_score(y_true, y_score)
        auroc_scores.append(auroc)
    mean_auroc = np.mean(auroc_scores)
    std_auroc = np.std(auroc_scores)

    return mean_auroc, std_auroc

def calculate_R_squared(GT, INF):
  constant_GT_mean = np.mean(GT, axis=0, keepdims=True).repeat(12320, axis=0)
  total_sum_of_square = np.sum((constant_GT_mean - GT) ** 2, axis=0)
  residual_sum_of_square = np.sum((INF - GT) ** 2, axis=0)
  r_square = 1 - (residual_sum_of_square / total_sum_of_square)
  mean_r_square = np.mean(r_square)
  std_r_square = np.std(r_square)
  return mean_r_square, std_r_square


In [ ]:
# Main block for calculating Overall Error, SCC, and Recall analysis

import warnings
warnings.simplefilter('ignore', category=RuntimeWarning)
with open("Level3/GT.pkl", 'rb') as f:
  GT = pickle.load(f)
index_all = pd.Index(load_gene_ids("geneid_12320.txt"), name='rid') # check your "geneid_12320.txt" path

output_df = []
for i in range(len(models)):
  fn = models.iloc[i,0]
  lmid = lmids.loc[(models.iloc[i,2], models.iloc[i,3]), 'lmid_list']
  print(f"model:{fn.replace('.pkl','')}, num(lm):{len(lmid)}")

  # Load test data from files
  with open("Level3/"+fn, "rb") as f:
      INF = pickle.load(f)

  # calculate Error and SCC
  err = calculate_Error(GT, INF)
  scc = calculate_SCC(GT, INF)
  pcc = calculate_PCC(GT, INF)
  smape = calculate_SMAPE(GT, INF)
  auroc_mean, auroc_std = calculate_AUROC(GT, INF)
  r_square_mean, r_square_std = calculate_R_squared(GT, INF)

  def recall_for_LM(GT,INF,lmid, outfn):
    # GT, INF: (12320, 176) numpy ndarray, whose order of rows (genes) matches with that of geneid_12320.txt
    # lmid: list of landmark ids
    # outfn: output text file name where the gene ids of BING (=landmark + inferred genes with Recall>0.95) are written
    nonlmid = index_all.drop(lmid).tolist()
    lm_sep_order = lmid + nonlmid

    GT_sep = pd.DataFrame(GT, index=index_all)
    INF_sep = pd.DataFrame(INF, index=index_all)
    GT_sep = GT_sep.loc[lm_sep_order,:].to_numpy()
    INF_sep = INF_sep.loc[lm_sep_order,:].to_numpy()
    r95, r99 = recall_test(GT_sep, INF_sep, lmid=lmid, nonlmid=nonlmid, num_lm=len(lmid), outfn=outfn)
    return r95, r99


  outfn = "BING_"+fn.replace(".pkl", "")
  r95, r99 = recall_for_LM(GT, INF, lmid, outfn)
  output_df.append([f"{np.mean(err):.4f}",f"{np.std(err):.4f}", f"{np.mean(scc):.4f}",f"{np.std(scc):.4f}",
                    f"{np.mean(pcc):.4f}",f"{np.std(pcc):.4f}", f"{np.mean(smape):.4f}",f"{np.std(smape):.4f}",
                    r95, r99,f"{auroc_mean:.4f}",f"{auroc_std:.4f}",f"{r_square_mean:.4f}",f"{r_square_std:.4f}"])

# write the result to a tab-delimited text file
output_df = pd.DataFrame(output_df, columns=["Error_mean","Error_stdev","SCC_mean","SCC_stdev","PCC_mean","PCC_stdev","SMAPE","SPE_stdev", "r95", "r99","AUROC_mean","AUROC_stdev","R2_mean","R2_stdev"])
recall_result =pd.concat([models, output_df], axis=1)
recall_result.to_csv("recall_result.txt", sep="\t", index=False)
# This result file is saved in the Google Colab local storage. Download manually or copy to Google Drive.


model:GNN_all_970, num(lm):970
Among 12320 inferred genes:
Rgene>0.95: 11867 (96.32%)
Rgene>0.99: 11366 (92.26%)
model:GNN_greedy_1, num(lm):1
Among 12320 inferred genes:
Rgene>0.95: 3347 (27.17%)
Rgene>0.99: 1050 (8.52%)
model:GNN_greedy_10, num(lm):10
Among 12320 inferred genes:
Rgene>0.95: 11643 (94.50%)
Rgene>0.99: 11001 (89.29%)
model:GNN_greedy_108, num(lm):108
Among 12320 inferred genes:
Rgene>0.95: 11990 (97.32%)
Rgene>0.99: 11416 (92.66%)
model:GNN_LASSO_1, num(lm):1
Among 12320 inferred genes:
Rgene>0.95: 11472 (93.12%)
Rgene>0.99: 10655 (86.49%)
model:GNN_LASSO_10, num(lm):10
Among 12320 inferred genes:
Rgene>0.95: 11439 (92.85%)
Rgene>0.99: 10602 (86.06%)
model:GNN_LASSO_108, num(lm):108
Among 12320 inferred genes:
Rgene>0.95: 11995 (97.36%)
Rgene>0.99: 11388 (92.44%)
model:GNN_random_1, num(lm):1
Among 12320 inferred genes:
Rgene>0.95: 3807 (30.90%)
Rgene>0.99: 1164 (9.45%)
model:GNN_random_10, num(lm):10
Among 12320 inferred genes:
Rgene>0.95: 11200 (90.91%)
Rgene>0.99: 1

In [ ]:
# Level 3 to Level 4
def level3_to_level4(fn, inputpath, outpath, estimate=False):
  def rzs(mat): # calculates robust z-score
    medians = mat.median(axis=1)
    sub = mat.subtract(medians, axis='index')
    mads = abs(sub).median(axis=1)
    # if estimate option is used, change the min_mad to the 1st percentile of mads if the value is larger than 0.01
    if estimate:
      min_mad = max(np.nanpercentile(mads, 1), 0.01)
    else:
      min_mad = 0.01
    mads = mads.clip(lower=min_mad)
    zscore_df = sub.divide(mads * 1.4826, axis='index')
    # return zscore_df.round(4)
    return zscore_df

  with open(inputpath+fn, 'rb') as f:
    arr = pickle.load(f)  # input pickle files of shape (12320, 176) numpy ndarray; row order should be same as gctx files
  rid = load_gene_ids("geneid_12320.txt") # check your "geneid_12320.txt" path
  df = pd.DataFrame(arr, index=rid)
  df = rzs(df)
  if not outpath.endswith("/"): outpath = outpath + "/"
  if not os.path.exists(outpath): os.makedirs(outpath)
  with open(outpath+fn, 'wb') as f:
    pickle.dump(df, f)

inputpath = "Level3/"
outpath = "Level4/"
for fn in os.listdir(inputpath):
  if fn.endswith(".pkl"):
    level3_to_level4(fn, inputpath, outpath)

In [12]:
# Rank the Level 4 data before computing gene rank correlation

l4path = "Level4/"
rdfpath = "ranked_df/"
if not os.path.exists(rdfpath): os.makedirs(rdfpath)
for fn in os.listdir(l4path):
  if fn.endswith(".pkl"):
    with open(l4path+fn, 'rb') as f:
      df = pickle.load(f)
    ranked_df = df.rank(axis=0)
    with open(rdfpath+fn, "wb") as f:
      pickle.dump(ranked_df, f)

In [13]:
# Main block for calculating Gene rank correlation
import scipy.stats as stats
rdfpath = "ranked_df/"

# Load Ground Truth
with open(rdfpath+"GT.pkl", "rb") as f:
  GT = pickle.load(f)

ranked_df_list = os.listdir(rdfpath)
lst = []
for fn in ranked_df_list:
  if not fn.endswith(".pkl"): continue
  with open(rdfpath+fn, "rb") as f:
    INF = pickle.load(f)

  # compare ranks in AIG
  corr_all = []
  for i in range(GT.shape[1]):
    gti = GT.iloc[:,i].to_numpy()
    infi = INF.iloc[:,i].to_numpy()
    corr_all.append(np.corrcoef(gti, infi)[0,1])

  # compare ranks in BING

  bingfn = "BING/BING_" + fn.replace(".pkl", "_recall_result.csv")
  bing_file = pd.read_csv(bingfn, header=0)
  bing_ids = bing_file[bing_file.iloc[:, 4] == 1].iloc[:, 1].astype(str).tolist()
  GT_bing = GT.loc[bing_ids,:]
  GT_bing = GT_bing.rank(axis=0)
  INF_bing = INF.loc[bing_ids,:]
  INF_bing = INF_bing.rank(axis=0)

  bing_ids99 = bing_file[bing_file.iloc[:, 5] == 1].iloc[:, 1].astype(str).tolist()
  GT_bing99 = GT.loc[bing_ids99,:]
  GT_bing99 = GT_bing99.rank(axis=0)
  INF_bing99 = INF.loc[bing_ids99,:]
  INF_bing99 = INF_bing99.rank(axis=0)

  bing95_no_of_lm = bing_file[bing_file.iloc[:, 4] == 1].iloc[:, 6].sum()
  bing99_no_of_lm = bing_file[bing_file.iloc[:, 5] == 1].iloc[:, 6].sum()



  corr_bing = []
  corr_bing99 = []
  for i in range(GT_bing.shape[1]):
    gti = GT_bing.iloc[:,i].to_numpy()
    infi = INF_bing.iloc[:,i].to_numpy()
    corr_bing.append(np.corrcoef(gti, infi)[0,1])

    gti99 = GT_bing99.iloc[:,i].to_numpy()
    infi99 = INF_bing99.iloc[:,i].to_numpy()
    corr_bing99.append(np.corrcoef(gti99, infi99)[0,1])

  print(f"{fn}\t{GT.shape[0]}\t{np.mean(corr_all):.4f}+-{np.std(corr_all):.4f}\t{GT_bing.shape[0]}\t{np.mean(corr_bing):.4f}+-{np.std(corr_bing):.4f}")
  lst.append([fn.replace(".pkl",""), GT.shape[0], f"{np.mean(corr_all):.4f}",f"{np.std(corr_all):.4f}",
              GT_bing.shape[0], f"{np.mean(corr_bing):.4f}",f"{np.std(corr_bing):.4f}", bing95_no_of_lm,
              GT_bing99.shape[0], f"{np.mean(corr_bing99):.4f}",f"{np.std(corr_bing99):.4f}", bing99_no_of_lm ])

# write the result to a tab-delimited text file
lst_df = pd.DataFrame(lst, columns=["model","num_AIG","rank_corr_AIG_mean","rank_corr_AIG_stdev",
                                             "num_BING","rank_corr_BING_mean","rank_corr_BING_stdev","bing95_lm",
                                             "num_BING_99", "rank_corr_BING99_mean","rank_corr_BING99_stdev","bing99_lm"])
lst_df.to_csv("Gene_Rank_Corr.txt", sep="\t", index=False)
# This result file is saved in the Google Colab local storage. Download manually or copy to Google Drive.

LR_greedy_108.pkl	12320	0.7640+-0.0876	11260	0.7921+-0.0835
GNN_greedy_1.pkl	12320	0.1698+-0.2963	3347	0.2829+-0.3489
MLP_random_324.pkl	12320	0.7647+-0.0929	11294	0.7854+-0.0896
LR_LASSO_108.pkl	12320	0.7253+-0.0997	11137	0.7564+-0.0958
GNN_LASSO_108.pkl	12320	0.8248+-0.0840	11995	0.8301+-0.0838
LR_all_970.pkl	12320	0.7657+-0.0768	11237	0.7959+-0.0731
RNAseqKNR_all_970.pkl	12320	0.1769+-0.1687	2437	0.3792+-0.2349
GNN_all_970.pkl	12320	0.8612+-0.0586	11867	0.8673+-0.0580
SwinIR_random_486.pkl	12320	0.6061+-0.1134	9526	0.6905+-0.1127
SwinIR_random_108.pkl	12320	0.5966+-0.1386	9527	0.6790+-0.1438
MLP_greedy_108.pkl	12320	0.8037+-0.0750	11415	0.8222+-0.0723
GNN_random_1.pkl	12320	0.1668+-0.2431	3807	0.2535+-0.2783
MLP_all_970.pkl	12320	0.8078+-0.0782	11479	0.8197+-0.0767
LR_random_486.pkl	12320	0.7659+-0.0803	11206	0.7962+-0.0761
MLP_random_486.pkl	12320	0.7710+-0.0894	11306	0.7923+-0.0857
GNN_greedy_10.pkl	12320	0.7585+-0.1582	11643	0.7663+-0.1586
SwinIR_random_324.pkl	12320	0.5563+-0.11